In [1]:
pip install transformers accelerate -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.7 MB/s eta 0:00:00


In [2]:
from sklearn.metrics import  accuracy_score, f1_score, confusion_matrix
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
import torch
import pandas as pd
import numpy as np

In [3]:
def tokenize_texts(tokenizer, texts):
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=128)
    return encodings


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
model_path = "/content/drive/MyDrive/IRproject/saved_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [6]:


# List of texts to classify
texts = [
    '''
    به گزارش "ورزش سه"، تیم ملی والیبال ایران دیروز برای حضور در لیگ ملت های 2021 از طریق دوحه راهی کشور ایتالیا شد، اما در همین فرودگاه امام خمینی تهران اتفاقات عجیبی برای سرپرست تیم ملی رخ داد.


 ماجرا از این قرار بود که امیر خوش خبر به خاطر سفر طولانی مدت تیم ملی مبالغ ارزی زیادی را با خود حمل می کرد. گویا سرپرست تیم ملی مجوزی هم از بانک مرکزی نداشته و به همین خاطر با سخت گیری ماموران مواجه شده است.


 میثم جعفری، مدیر تشریفات فدراسیون والیبال در این باره می گوید: «متاسفانه به دلیل برخی ناهماهنگی ها و البته تاخیر بانک مرکزی مجوزهای لازم به دست خوش خبر نرسید و به همین دلیل ماموران مانع خروج وی از کشور شدند. پس از رایزنی های طولانی سرانجام مقرر شد تا من به عنوان نماینده فدراسیون و جایگزین امیر خوش خبر در فرودگاه بمانم تا سرپرست تیم ملی بتواند همراه با تیم به ایتالیا سفر کند.»


 البته که در نهایت خیلی از این یوروها در تهران ماند و خوش خبر با مبلغ کمتری به ایتالیا رفت. البته فدراسیون والیبال در تلاش است تا با کسب مجوز لازم، طی روزهای اینده مابقی مبلغ به ایتالیا منتقل شود.

    ''',
    '''
    تصاویر| زخم ناسور زباله بر گونه سبز روستای کشل ازادسرا خودنمایی کوه زباله در روستای سرسبز «کشل ازادسرا» استانه اشرفیه به عنوان زخمی چرکین بر چهره سرسبز این روستا، ساکنان ان را با مشکلات عدیده ای مواجه ساخته است. افتاب نیوز :
روستای کشل ازادسرا استانه اشرفیه در گیلان این روزها با زخمی چرکین بر چهره طبیعت بکر خود دست و پنجه نرم می کند و ساکنان این روستای گردشگر پذیر را با مشکلات عدیده ای مواجه ساخته و مردم منطقه همچنان در انتظار توجه ویژه مسیولان به این روستا هستند.

    '''
]

# Tokenize the texts
encodings = tokenizer(texts, truncation=True, padding=True, max_length=128, return_tensors='pt')

# Move the input tensors to the device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_ids = encodings["input_ids"].to(device)
attention_mask = encodings["attention_mask"].to(device)

model.to(device)

# Make predictions
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)

# Move the logits to the CPU and convert to NumPy array
logits = outputs.logits.detach().cpu().numpy()

# Get the predicted class labels
predicted_labels = np.argmax(logits, axis=1)

# Map the predicted labels to class names
class_names = ['Beinolmelal', 'Eghtesadi', 'Ejtemaee', 'ElmiVaDaneshghai', 'FarhangHonarVaResane', 'Siasi', 'Varzeshi']
predicted_classes = [class_names[label] for label in predicted_labels]

# Print the results
for text, predicted_class in zip(texts, predicted_classes):
    # print(f"Text: {text}")
    print(f"Predicted Class: {predicted_class}")
    print()


Predicted Class: Varzeshi

Predicted Class: Ejtemaee

